# A blogging workflow with Jupyter and Wordpress

Abstract: How to convert your Jupyter notebook as blog post, including all the Wordpress features, plots, custom formatting, and latex formulas.

For me, [Jupyter](http://jupyter.org/) is a ready made solution for two problems simultaneously. It's a great tool for prototyping all sorts of algorithms in the [SciPy](https://www.scipy.org/) environment, while it also does fantastic job in taking up notes on the mathematics behind those algorithms thanks to Markdown and the beautiful formatting of mathematical formulas using the Latex syntax.

To publish a Jupyter notebook, you can simply upload the .ipynb file to GitHub, which renders as a static page as you open it there like [this][this]. However, when writing a blog, you would like to have your own site style, and more site structure and features, other than just dropping individual notebooks here and there. 

To easily maintain a blogging site with styles, site structure, and comments, there's [Wordpress](https://wordpress.com/). In the following, I will explain my attempt to translate a Jupyter notebook to a post in Wordpress with minimal manual labour. This post by [Mian Chen][mianchen] helped me a lot.

[this]: [https://github.com/mattipellikka/blog/blob/master/how_to_wordpress/how_to_wordpress.ipynb]
[mianchen]: http://www.mianchen.com/wordpress-blogging-with-jupyter-notebook-in-five-simple-steps/

In [1]:
# preample
import numpy as np
import matplotlib.pyplot as plt; 
from IPython.display import display
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')